<h2 align="center">Парсинг музыки Yandex.Music</h2> 

Загрузим все песни по 10 жанрам в единый словарь с ключом - исполнителем и значением - песней

### 1.Загружаем библиотеки

In [ ]:
import requests
from bs4 import BeautifulSoup

### 2. Загружаем URL плейлистов по нужным жанрам

In [7]:
urls_yandex = {
    'blues':'https://music.yandex.ru/users/music-blog/playlists/2731',
    'classical':'https://music.yandex.ru/users/yamusic-top/playlists/1138',
    'country':'https://music.yandex.ru/users/ya.musicgenre/playlists/1040',
    'disco':'https://music.yandex.ru/users/music.partners/playlists/2061',
    'hip hop':'https://music.yandex.ru/users/yamusic-top/playlists/1039',
    'jazz':'https://music.yandex.ru/users/ya.musicgenre/playlists/1125',
    'metal':'https://music.yandex.ru/users/yamusic-top/playlists/1080',
    'pop':'https://music.yandex.ru/users/yamusic-top/playlists/1010',
    'reggae':'https://music.yandex.ru/users/yamusic-top/playlists/1034',
    'rock':'https://music.yandex.ru/users/yamusic-top/playlists/1013'
}
songs_yandex = dict(zip(urls_yandex.keys(),[{} for x in range(10)]))

### 3. Парсим плейлисты

In [63]:
from tqdm import tqdm
for genre,url in tqdm(zip(urls_yandex.keys(),urls_yandex.values())):
    response = requests.get(url)
    tree = BeautifulSoup(response.text, features="html.parser")
    song_names = tree.find_all('div', {'class': 'd-track__name'})
    song_artists = tree.find_all('div', {'class': 'd-track__meta'})
    song_names = BeautifulSoup(str(song_names))
    songs = dict()
    info_names = song_names.find_all('div')
    info_artists = song_artists
    for name,artist in zip(info_names,info_artists):
        songs[name['title']] = artist.find('a')['title']
    songs_yandex[genre] = songs

10it [00:10,  1.06s/it]


In [64]:
[[len(list(songs_yandex.values())[i]),list(songs_yandex.keys())[i]] for i in range(10)]

[[100, 'blues'],
 [100, 'classical'],
 [20, 'country'],
 [35, 'disco'],
 [100, 'hip hop'],
 [19, 'jazz'],
 [100, 'metal'],
 [100, 'pop'],
 [99, 'reggae'],
 [99, 'rock']]

### 4. Загружаем в JSON

In [64]:
import json
with open('songs_yandex.json','w') as f:
    json.dump(songs_yandex,f)

---

<h2 align="center">Загрузка музыки songs_yandex</h2> 

Следующее, что нужно, это выгрузить все песни из словаря __songsyandex__ \
Сделаем это через библиотеку yandex-music

In [7]:
from yandex_music import Client
import os
import shutil


In [8]:
token = 'Вставьте токен Yandex.Music'

In [9]:
client = Client(token).init()

Yandex Music API v2.0.1, Copyright (C) 2019-2022 Il`ya (Marshal) <https://github.com/MarshalX>
Licensed under the terms of the GNU Lesser General Public License v3 (LGPLv3)



In [8]:
c=0
for genre in songs_yandex:
    os.mkdir(genre)
    for artist, song in (zip(songs_yandex[genre].values(), songs_yandex[genre].keys())):
        searcher = client.search(artist+' '+song).best.result
        if genre != 'classical':
            searcher.download(os.path.abspath(genre)+'/'+searcher.title+'.wav')
        else:
            c+=1
            searcher.download(os.path.abspath(genre)+'/'+str(c)+'.wav')
    print(genre,' - COMPELETED')

FileExistsError: [Errno 17] File exists: 'blues'

In [72]:
for genre in ['pop','reggae','rock']:
    os.mkdir(genre)
    for artist, song in (zip(songs_yandex[genre].values(), songs_yandex[genre].keys())):
        searcher = client.search(artist+' '+song).best.result
        try:
            searcher.download(os.path.abspath(genre)+'/'+searcher.title+'.wav')
        except:
            pass
    print('COMPLETED')

COMPLETED
COMPLETED
COMPLETED


In [ ]:
os.mkdir('rock')
for artist, song in (zip(songs_yandex[genre].values(), songs_yandex[genre].keys())):
    searcher = client.search(artist+' '+song).best.result
    try:
        searcher.download(os.path.abspath(genre)+'/'+searcher.title+'.wav')
    except:
        pass
print('COMPLETED')

#### Как все происходило?

Переменная __token__ - это токен аккаунта Yandex.Music, который нужно получить для работы с библиотекой. После инициализации Client мы можем выполнять множество действий, но займемся одним - поиском нужных песен через __.search__ \
После того как трек найден, берем лучший результат и скачиваем его в формате __.wav__

<h2 align="center">Парсинг музыки Spotify</h2> 

**Зачем это нужно?** \
В парсинге Яндекс Музыки мы получили месево из русских/английских песен, непонятно, как это отразится на классификации. Чтобы защититься, спарсим еще и Spotify, где, во-первых, будут плейлисты с 100 песен(в целом можно подшаманить с Яндекс Музыкой и поделить пополам песни в тех жанрах, где песен меньше 100), и, во-вторых, будут песни лишь на английском

In [3]:
urls_spotify = {
    'blues': 'https://open.spotify.com/playlist/37i9dQZF1DXd9rSDyQguIk',
    'classical': 'https://open.spotify.com/playlist/37i9dQZF1DWWEJlAGA9gs0',
    'country': 'https://open.spotify.com/playlist/1mJhSx6aYQmINsZ8dG4gzU',
    'disco': 'https://open.spotify.com/playlist/27wJlDQft43Y0b8uAZ73rk',
    'hip hop': 'https://open.spotify.com/playlist/3RcRK9HGTAm9eLW1LepWKZ',
    'jazz': 'https://open.spotify.com/playlist/37i9dQZF1DXbITWG1ZJKYt',
    'metal': 'https://open.spotify.com/playlist/37i9dQZF1DWWOaP4H0w5b0',
    'pop': 'https://open.spotify.com/playlist/6mtYuOxzl58vSGnEDtZ9uB',
    'reggae': 'https://open.spotify.com/playlist/37i9dQZF1DXbSbnqxMTGx9',
    'rock': 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U',
}
urls_spotify_50 = {
    'blues': 'https://open.spotify.com/playlist/37i9dQZF1EQpz3DZCEoX3g',
    'classical': 'https://open.spotify.com/playlist/37i9dQZF1EQn1VBR3CMMWb',
    'country': 'https://open.spotify.com/playlist/37i9dQZF1EQmPV0vrce2QZ',
    'disco': 'https://open.spotify.com/playlist/37i9dQZF1DX2H07nGW2AbS',
    'hip hop': 'https://open.spotify.com/playlist/37i9dQZF1EQnqst5TRi17F',
    'jazz': 'https://open.spotify.com/playlist/37i9dQZF1EQqA6klNdJvwx',
    'metal': 'https://open.spotify.com/playlist/37i9dQZF1EQpgT26jgbgRI',
    'pop': 'https://open.spotify.com/playlist/37i9dQZF1EQncLwOalG3K7',
    'reggae': 'https://open.spotify.com/playlist/37i9dQZF1EQpjs4F0vUZ1x',
    'rock': 'https://open.spotify.com/playlist/37i9dQZF1EQpj7X7UK8OOF',
}

Нашли какие-то плейлисты со Spotify с разным количеством треком(>/< 100). urls_spotify_50 - миксы по 50 треков

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import warnings
warnings.filterwarnings('ignore')

In [4]:
client_id = 'Client_id для Spotify'
secret = 'secret для Spotify'
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=secret)
spotify = spotipy.Spotify(auth_manager=auth_manager)
songs_spotify_50 = dict(zip(urls_spotify_50.keys(),[{} for x in range(10)]))
songs_spotify = dict(zip(urls_spotify_50.keys(),[{} for x in range(10)]))

In [5]:
for genre in (urls_spotify_50):
    songs = dict()
    play = spotify.playlist(urls_spotify_50[genre])
    for i in range(50):
        artist = play['tracks']['items'][i]['track']['artists'][0]['name']
        song = play['tracks']['items'][i]['track']['name']
        songs[song] = artist
    songs_spotify_50[genre] = songs

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


In [6]:
for genre in (urls_spotify):
    songs = dict()
    play = spotify.playlist(urls_spotify[genre])
    for i in range(50):
        artist = play['tracks']['items'][i]['track']['artists'][0]['name']
        song = play['tracks']['items'][i]['track']['name']
        songs[song] = artist
    songs_spotify[genre] = songs

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


In [19]:
c=0
for genre in songs_spotify_50:
#     os.mkdir(genre+'_spotify')
    for artist, song in (zip(songs_spotify_50[genre].values(), songs_spotify_50[genre].keys())):
        try:
            searcher = client.search(artist+' '+song).best.result
            if genre != 'classical':
                try:
                    searcher.download(os.path.abspath(genre+'_spotify')+'/'+searcher.title+'.wav')
                except:
                    pass
            else:
                c+=1
                try:
                    searcher.download(os.path.abspath(genre+'_spotify')+'/'+str(c)+'.wav')
                except:
                    pass
        except:
            pass
    print(genre,' - COMPLETED')

blues  - COMPLETED
classical  - COMPLETED
country  - COMPLETED
disco  - COMPLETED
hip hop  - COMPLETED
jazz  - COMPLETED
metal  - COMPLETED
pop  - COMPLETED
reggae  - COMPLETED
rock  - COMPLETED


In [7]:
track_urls = dict(zip(urls_spotify_50.keys(),[{} for x in range(10)]))

In [8]:
features = dict(zip(urls_spotify_50.keys(),[{} for x in range(10)]))

In [9]:
for genre in songs_spotify_50:
    songs = dict()
    play = spotify.playlist(urls_spotify_50[genre])
    for i in range(50):
        urling = play['tracks']['items'][i]['track']['external_urls']
        artist = play['tracks']['items'][i]['track']['artists'][0]['name']
        song = play['tracks']['items'][i]['track']['name']
        songs[song] = list(urling.values())[0]
    track_urls[genre] = songs

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


In [11]:
warnings.filterwarnings("ignore", category=FutureWarning)
from tqdm import tqdm
for genre in tqdm(songs_spotify_50):
    songs_url = dict()
    for track in list(track_urls[genre].keys()):
        feat = spotify.audio_features(track_urls[genre][track])[0]
        songs_url[track] = feat
    features[genre] = songs_url

  0%|                                                    | 0/10 [00:00<?, ?it/s]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to c

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at:

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at:

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at:

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 90%|███████████████████████████████████████▌    | 9/10 [01:26<00:09,  9.45s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to c

In [14]:
import json
with open('features_songs_spotify.json','w') as f:
    json.dump(features,f)